In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [3]:
import warnings; warnings.simplefilter('ignore')
import gensim as gen
import json
import codecs
import regex as re
import os.path
from scipy import spatial

import local_settings
import django
django.setup()

from sefaria.model import *
from sefaria.system.exceptions import InputError, PartialRefInputError

WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/blockspeiser/Sefaria-Project/wiki/Regular-Expression-Engines
WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/Sefaria/Sefaria-Project/wiki/Regular-Expression-Engines


In [3]:
import hebrew_spellcheck

In [4]:
word_expander = hebrew_spellcheck.word_expander

In [36]:
stopwords = codecs.open('./hebrew_stopwords.txt', encoding='utf8').read().strip().split('\n')
stopwords_regex = u"(?:\s|^)({})(?=\s|$)".format(u"|".join(stopwords))
stopwords_regex = re.compile(stopwords_regex)

In [37]:
phrases = codecs.open('./select_phrases.txt', encoding='utf8').read().strip().split('\n')
phrases_regex = u"(\s|^)({})(?=\s|$)".format(u"|".join(phrases))
phrases_regex = re.compile(phrases_regex)

In [38]:
def get_talmud_topic_ranged():
    """
    Reads every file in the sugyot directory.  
    Each file in this directory contains semantically meaningful ranged refs for Talmud
    :return: List of all Talmudic Ranged Refs
    """
    all_gemara_topic_ranges = []
    for filename in os.listdir('./sugyot/'):
        with codecs.open('./sugyot/'+filename, 'rb', encoding='utf8') as csvfile:
            reader = csv.reader(csvfile)
            reader.next()
            all_gemara_topic_ranges += [row[0] for row in reader]
    return all_gemara_topic_ranges


def create_list_off_talmud_books():
    """
    Uses Sefaria Project to create a list of titles of every Talmud Tractate
    :return: Set of all Talmud book titles
    """
    indexes = library.all_index_records()
    talmud_books = set(library.get_indexes_in_category("Bavli"))
    talmud_books_modified = {u'Avodah Zarah',  u'Bava Batra', u'Bava Kamma',
     u'Bava Metzia', u'Beitzah', u'Berakhot', u'Chagigah', u'Eruvin', u'Gittin', u'Horayot',
     u'Ketubot', u'Kiddushin', u'Makkot', u'Megillah', u'Menachot', u'Moed Katan',
     u'Nazir', u'Nedarim', u'Pesachim', u'Rosh Hashanah', u'Sanhedrin', u'Shabbat', u'Shevuot',
     u'Sotah', u'Sukkah', u'Taanit',  u'Yevamot', u'Yoma', u'Zevachim'}
    return talmud_books_modified


def create_list_off_tanakh_books():
    """
    Uses Sefaria Project to create a list of titles of every book in Tanakh 
    :return: Set of all Tanakh book titles
    """
    indexes = library.all_index_records()
    tanakh_books = set(library.get_indexes_in_category("Tanakh"))
    tanakh_books_modified = set([u'Judges', u'Deuteronomy', u'Genesis', u'Exodus',
                             u'Leviticus', u'II Kings', u'Joshua',
                             u'I Samuel', u'Numbers', u'I Kings', u'II Samuel'])
    return tanakh_books_modified
    


def get_tanakh_topic_ranges():
    """
    Creates semantically meaningful ranged refs for Tanakh using Herzog's breakdown
    :return: List of all Tanakh Ranged Refs
    """
    with codecs.open('level_3_wo_overlaps.json', 'r', encoding='utf8' ) as the_file:
        segs = json.load(the_file, encoding='utf8')
    tanakh_topic_ranged_refs = []

    for seg in segs:
        b_ref = seg['b_ref']
        e_ref = seg['e_ref']
        book = b_ref.rsplit(' ', 1)[0]
        b_chapter = b_ref.rsplit(' ', 1)[1].split(':')[0]
        b_verse = b_ref.rsplit(' ', 1)[1].split(':')[1]
        e_chapter = e_ref.rsplit(' ', 1)[1].split(':')[0]
        e_verse = e_ref.rsplit(' ', 1)[1].split(':')[1]

        verses = "{}:{}-{}".format(b_chapter, b_verse, e_verse) if b_chapter == e_chapter else "{}:{}-{}:{}".format(b_chapter, b_verse, e_chapter, e_verse)
        tanakh_topic_ranged_refs.append("{} {}".format(book, verses))
    
    return tanakh_topic_ranged_refs
    

def segment_range_dicts(topic_ranged_refs):
    """
    Creates two dictionaries.  These are mappings between ranged refs and segments refs.  
    This receives a list of ranged refs.  It iterates over every ranged ref and creates a mapping between the ranged refs and the segment refs.
    :param topic_ranged_refs: list of ranged refs
    :return: tuple of dicts. The first is dict mapping ranged refs to all segment refs.  The second is a dict mapping each segment ref to its corresponding ranged ref.
    """
    ranged_to_segment = {}
    segment_to_ranged = {}

    for topic_ranged_ref in topic_ranged_refs:
        ranged_to_segment[topic_ranged_ref] = {}
        topic_seg_refs = Ref(topic_ranged_ref).range_list()
        for seg_ref in topic_seg_refs:
            ranged_to_segment[topic_ranged_ref][seg_ref.normal()] = ""
            segment_to_ranged[seg_ref.normal()] = topic_ranged_ref
            
    return ranged_to_segment, segment_to_ranged

def is_from_category(ref, books_in_category):
    """
    Checks to see if a tref belongs to a list of book within a particular category
    For Example:  Is Genesis 12:3 within Tanakh
    :param ref: tref to be examined
    :param books_in_category: List of books in category
    :return: Boolean Value determining if Tref is within category
    """
    try:
        if Ref(ref).index.title in books_in_category:
            return True
        else:
            return False
    except InputError:
        return False
    

def pull_out_suffix(string):
    """
    Receives a string of Hebrew Text.  Iterates over every hebrew word and splits each
    word into its root and suffix.  
    :param string: String of Hebrew Text
    :return: String with Hebrew compound words split into Root and Suffix
    """
    string = string.split()
    string = ' '.join([word_expander.get(word, word) for word in string])
    return string

def remove_stopwords(string):
    """
    Replaces Stopwords with a space character  
    :param string: String of Hebrew Text
    :return: String without stopwords
    """
    return re.sub(stopwords_regex, u' ', string)

def remove_dicta_prefix(string):
    """
    Removes prefixes that were detected by Dicta
    :param string: String of Hebrew Text
    :return: String without prefixes
    """
    return re.sub(ur'[\u05d0-\u05ea]+┉', u'', string)

def remove_punctuation(data):
    """
    Removes various punctation from Hebrew text.
    :param data: String of Hebrew Text
    :return: String without punctuation
    """
    data = re.sub(ur'־', u' ', data)
    data = re.sub(ur'\([^)]+\)', u' ', data)
    data = re.sub(ur'<[^>]+>', u' ', data)
    data = re.sub(ur'\[[^\]]+\]', u' ', data)
    data = re.sub(ur'[^ \u05d0-\u05ea"\'״׳]', u' ', data)
    data = re.sub(ur'(^|\s)["\'״׳]+', u' ', data)
    data = re.sub(ur'["\'״׳]+(\s|$)', u' ', data)
    return data

def strip_stopwords_and_remove_punctuation(data):
    """
    This method takes a string of hebrew text and does all necessary cleaning for Word2Vec.
    :param data: String of Hebrew Text
    :return: String ready for Word2Vec model
    """
    data = data.strip().split(u'~~')[1]
    data = remove_dicta_prefix(data)
    data = remove_punctuation(data)
    data = pull_out_suffix(data)
    data = remove_stopwords(data)
    data = u' '.join(data.split())
    return data



def create_multiple_word_phrases(data):
    """
    Combines selected multiple word phrases with underscore. 
    For example:  New York ----> New_York
    This allows Word2Vec to handle multiple word phrases
    :param data: String of Hebrew Text
    :return: String with connected multiple word phrases
    """
    def connect_with_underscore(matchobj):
        return matchobj.group(1) + matchobj.group(2).replace(u" ", u"_")
    return re.sub(phrases_regex, connect_with_underscore, data)


def this_is_a_bad_line(data):
    """
    Checks to see if a line from the file should be included in the Word2Vec model.
    There are some lines within the Dicta Prefix file that we cannot or do not want to include.
    :param data: line of hebrew text from Dicta File
    :return: Boolean Value determining if this line valid
    """
    if u'~~' not in data:
        return True
    if data.strip().split(u'~~')[1].strip().startswith(u"<br><br><big><strong>הדרן עלך"):
        return True
    
    return False
    

def get_segments(filename):
    """
    Combs through the entire Sefaris Hebrew Library and cleans the text for Doc2Vec.
    Creates a dict:
        Key:  Ref
        Value:  The text of that ref cleaned and ready for Doc2Vec
    :param filename: Dicta Prefix Filename
    :return: Dict for Doc2Vec
    """
    
    all_tanakh_books = create_list_off_tanakh_books()
    all_talmud_books = create_list_off_talmud_books()
    
    all_data = {}
    
    
    tanakh_topic_ranged_refs = get_tanakh_topic_ranges()
    talmud_topic_ranged_refs = get_talmud_topic_ranged()
    
    tanakh_ranged_to_segment, tanakh_segment_to_ranged = segment_range_dicts(tanakh_topic_ranged_refs)
    talmud_ranged_to_segment, talmud_segment_to_ranged = segment_range_dicts(talmud_topic_ranged_refs)


    for index, data in enumerate(codecs.open(filename, encoding='utf8')):
        
        if this_is_a_bad_line(data):
            continue
            
        ref = data.split(u'~')[0]
        data = strip_stopwords_and_remove_punctuation(data)
        data = create_multiple_word_phrases(data)
        
          
        if index%100000 == 0:
            print index
            
        if is_from_category(ref, all_tanakh_books):
            this_ref_ranged_reg = tanakh_segment_to_ranged[ref]        
            tanakh_ranged_to_segment[this_ref_ranged_reg][ref] = data
        elif is_from_category(ref, all_talmud_books):
            this_ref_ranged_reg = talmud_segment_to_ranged[ref]
            talmud_ranged_to_segment[this_ref_ranged_reg][ref] = data
        else:
            all_data[ref] = data
            
    for text_ranged_ref in tanakh_topic_ranged_refs:
        object_ref = Ref(text_ranged_ref)
        all_text_subrefs = [seg_ref.normal() for seg_ref in object_ref.range_list()]
        all_verses = [tanakh_ranged_to_segment[text_ranged_ref][seg_ref] for seg_ref in all_text_subrefs]
        all_data[text_ranged_ref] = u' '.join(all_verses)
        
    for text_ranged_ref in talmud_topic_ranged_refs:
        object_ref = Ref(text_ranged_ref)
        all_text_subrefs = [seg_ref.normal() for seg_ref in object_ref.range_list()]
        all_verses = [talmud_ranged_to_segment[text_ranged_ref][seg_ref] for seg_ref in all_text_subrefs]
        all_data[text_ranged_ref] = u' '.join(all_verses)
    
    return all_data

In [39]:
segments = get_segments('./sefaria-export_prefix_refs.txt')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000


In [1]:
all_clean_docs_filename = 'cleaned_docs_for_doc2vec.txt'
with codecs.open(all_clean_docs_filename, 'wb', encoding='utf8') as the_file:
    for k, v in segments.items():
        the_file.write(u""+k+u"||||"+v+u"\n")
        

Initiate a SegmentGenerator object to pass to the model

In [30]:
class SegmentGenerator(object):
    def __init__(self, segments_filename):
        self.segments_filename = segments_filename

    def __iter__(self):
        with codecs.open(self.segments_filename, 'rb', encoding='utf8') as the_file:
            for line in the_file:
                ref = line.split(u"||||")[0]
                data = line.split(u"||||")[1]
                yield gen.models.doc2vec.TaggedDocument(gen.utils.simple_preprocess(data), [ref])

In [31]:
segments_generator = SegmentGenerator(all_clean_docs_filename)

### Model intiation

In [32]:
model = gen.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=40, dm=0, dbow_words=1)

Now we have initiated the Word2Vec model. The next step is to call the **build_vocab** method for the preliminary scan of the text. Call the method and print how many words are in the vocabulary of our text

In [33]:
model.build_vocab(segments_generator)
print("Number of words in vocabulary: {}".format(len(model.wv.vocab)))

INFO collecting all words and their counts
INFO PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO PROGRESS: at example #10000, processed 480183 words (493516/s), 29343 word types, 10000 tags
INFO PROGRESS: at example #20000, processed 951957 words (444592/s), 39961 word types, 20000 tags
INFO PROGRESS: at example #30000, processed 1425416 words (464963/s), 48038 word types, 30000 tags
INFO PROGRESS: at example #40000, processed 1894717 words (512670/s), 54567 word types, 40000 tags
INFO PROGRESS: at example #50000, processed 2365377 words (501974/s), 60171 word types, 50000 tags
INFO PROGRESS: at example #60000, processed 2823417 words (497557/s), 64944 word types, 60000 tags
INFO PROGRESS: at example #70000, processed 3297189 words (506217/s), 69025 word types, 70000 tags
INFO PROGRESS: at example #80000, processed 3766242 words (410361/s), 73037 word types, 80000 tags
INFO PROGRESS: at example #90000, processed 4241438 words (506678/s), 76961 word types, 900

INFO PROGRESS: at example #800000, processed 37611106 words (377172/s), 194861 word types, 800000 tags
INFO PROGRESS: at example #810000, processed 38059767 words (388978/s), 195819 word types, 810000 tags
INFO PROGRESS: at example #820000, processed 38515852 words (259422/s), 196805 word types, 820000 tags
INFO PROGRESS: at example #830000, processed 38986308 words (354775/s), 197946 word types, 830000 tags
INFO PROGRESS: at example #840000, processed 39459622 words (365770/s), 198984 word types, 840000 tags
INFO PROGRESS: at example #850000, processed 39914003 words (392453/s), 199844 word types, 850000 tags
INFO PROGRESS: at example #860000, processed 40391398 words (438661/s), 200825 word types, 860000 tags
INFO PROGRESS: at example #870000, processed 40861805 words (459527/s), 201860 word types, 870000 tags
INFO PROGRESS: at example #880000, processed 41321675 words (402163/s), 203070 word types, 880000 tags
INFO PROGRESS: at example #890000, processed 41820841 words (486125/s), 2

INFO deleting the raw counts dictionary of 259021 items
INFO sample=0.001 downsamples 24 most-common words
INFO downsampling leaves estimated 71459864 word corpus (98.1% of prior 72845375)
INFO estimated required memory for 151713 words and 100 dimensions: 1127306700 bytes
INFO resetting layer weights


Number of words in vocabulary: 151713


In the previous step we created the vocabulary for our model, it is now time to train! Don't forget to add the following parameters:
1. total_examples=model.corpus_count 
2. epochs=model.epochs

In [34]:
model.train(segments_generator, total_examples=model.corpus_count, epochs=10)

INFO training model with 3 workers on 151713 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
INFO EPOCH 1 - PROGRESS: at 0.13% examples, 94930 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 0.30% examples, 104231 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 0.49% examples, 115582 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 0.70% examples, 120362 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 0.90% examples, 123805 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 1.11% examples, 126287 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 1.31% examples, 127989 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 1.50% examples, 128699 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 1.71% examples, 130050 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 1.91% examples, 131641 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 2.09% exam

INFO EPOCH 1 - PROGRESS: at 18.38% examples, 131651 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 18.57% examples, 131715 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 18.76% examples, 131764 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 18.95% examples, 131835 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 19.15% examples, 131887 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 19.36% examples, 132015 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 19.55% examples, 132038 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 19.73% examples, 132024 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 19.95% examples, 132070 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 20.16% examples, 132135 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 20.37% examples, 132188 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 20.57% examples, 132234 words/s, in_q

INFO EPOCH 1 - PROGRESS: at 37.72% examples, 134598 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 37.92% examples, 134633 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 38.13% examples, 134657 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 38.32% examples, 134666 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 38.51% examples, 134708 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 38.71% examples, 134723 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 38.92% examples, 134731 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 39.12% examples, 134737 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 39.33% examples, 134778 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 39.55% examples, 134766 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 39.77% examples, 134793 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 39.98% examples, 134819 words/s, in_q

INFO EPOCH 1 - PROGRESS: at 57.13% examples, 135872 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 57.29% examples, 135844 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 57.48% examples, 135858 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 57.69% examples, 135852 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 57.90% examples, 135876 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 58.11% examples, 135884 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 58.31% examples, 135881 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 58.53% examples, 135880 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 58.73% examples, 135903 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 58.94% examples, 135919 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 59.15% examples, 135905 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 59.37% examples, 135938 words/s, in_q

INFO EPOCH 1 - PROGRESS: at 76.36% examples, 136216 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 76.55% examples, 136211 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 76.74% examples, 136217 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 76.95% examples, 136240 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 77.17% examples, 136250 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 77.37% examples, 136274 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 77.58% examples, 136303 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 77.79% examples, 136320 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 77.99% examples, 136335 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 78.21% examples, 136353 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 78.30% examples, 136185 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 1 - PROGRESS: at 78.50% examples, 136206 words/s, in_q

...

INFO EPOCH 10 - PROGRESS: at 18.60% examples, 135979 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 18.80% examples, 135928 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 18.97% examples, 135954 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 19.17% examples, 135982 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 19.37% examples, 136045 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 19.57% examples, 136065 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 19.76% examples, 136072 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 19.96% examples, 136066 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 20.16% examples, 136055 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 20.36% examples, 136049 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 20.55% examples, 136119 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 20.75% examples, 136093 w

INFO EPOCH 10 - PROGRESS: at 37.74% examples, 136981 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 37.94% examples, 136986 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 38.14% examples, 136977 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 38.33% examples, 137000 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 38.53% examples, 137015 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 38.71% examples, 137017 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 38.92% examples, 137057 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 39.10% examples, 137014 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 39.31% examples, 137035 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 39.52% examples, 137031 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 39.74% examples, 137020 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 39.95% examples, 137035 w

INFO EPOCH 10 - PROGRESS: at 56.71% examples, 136798 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 56.91% examples, 136798 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 57.11% examples, 136810 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 57.29% examples, 136813 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 57.48% examples, 136773 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 57.68% examples, 136751 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 57.84% examples, 136659 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 58.01% examples, 136628 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 58.21% examples, 136646 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 58.39% examples, 136608 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 58.60% examples, 136606 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 58.81% examples, 136621 w

INFO EPOCH 10 - PROGRESS: at 75.06% examples, 135516 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 75.28% examples, 135520 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 75.47% examples, 135541 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 75.67% examples, 135550 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 75.85% examples, 135555 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 76.04% examples, 135554 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 76.24% examples, 135565 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 76.43% examples, 135575 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 76.64% examples, 135580 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 76.84% examples, 135580 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 77.05% examples, 135590 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 77.26% examples, 135598 w

INFO EPOCH 10 - PROGRESS: at 93.97% examples, 136098 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 94.16% examples, 136093 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 94.34% examples, 136090 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 94.53% examples, 136080 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 94.73% examples, 136085 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 94.94% examples, 136098 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 95.13% examples, 136091 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 95.34% examples, 136109 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 95.54% examples, 136111 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 95.72% examples, 136107 words/s, in_qsize 5, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 95.90% examples, 136096 words/s, in_qsize 6, out_qsize 0
INFO EPOCH 10 - PROGRESS: at 96.12% examples, 136092 w

In [35]:
test_topics = [u"פסח", u"תפילה", u"משה", u"אברהם", u"צדקה", u"שבת", u"רות", u"אסתר", u"תשובה", u"חטא", u"חגים", u"קהילה", u"ישראל", u"צדק", u"התגלות", u"אהבה", u"מין", u"תפילין", u"סביבה"]

In [38]:
def get_ref_score(ref):
    score = 0
    try:
        score = 1-spatial.distance.cosine(model.docvecs[ref], model[u'פסח']) 
    except KeyError:
        pass
    return score


test_topics_dict = {}
for topic in test_topics:
    test_topics_dict[topic] = {}
    topic_sources = model.docvecs.most_similar([model[topic]], topn=3000)
    topic_sources = [x[0] for x in topic_sources]
    set_of_related = set(topic_sources)
    for tref in topic_sources:
        try:
            oref = Ref(tref)
        except PartialRefInputError:
            continue
        for l in oref.linkset():
            for other_ref in l.refs:
                if other_ref != tref:
                    set_of_related.add(other_ref)
    print len(set_of_related)
    final_scores = {}
    for tref in set_of_related:
        try:
            if not Ref(tref).is_segment_level():
                continue
        except InputError:
            continue

        final_scores[tref] = get_ref_score(tref)
        try:
            oref = Ref(tref)
        except InputError:
            print tref
            continue
        for l in oref.linkset():
            for other_ref in l.refs:
                if other_ref != tref:
                    try:
                        other_oref = Ref(other_ref)
                    except InputError:
                        continue
                    if other_oref.is_segment_level():
                        final_scores[tref] += get_ref_score(other_ref)
    final_scores = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    for top_source in final_scores[:100]:
        test_topics_dict[topic][top_source[0]] = top_source[1]   
        

6645
6100
5354
7285
6111
6109
7745
7344
3993
5056
6052
3521
5853
8124
4154
5105
5900
6217
7724


In [39]:
with codecs.open('test_topics.json', 'w', encoding='utf8') as the_file:
    json.dump(test_topics_dict, the_file, indent=2, ensure_ascii=False)   